<a href="https://colab.research.google.com/github/wshuyi/demo_inference_ulmfit_fastai/blob/master/demo_fastai_inference_part1_train_and_valid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from fastai.text import *

In [2]:
!wget https://github.com/wshuyi/my_public_datasets/raw/master/yelp_sample_balanced.zip

--2019-07-21 21:04:25--  https://github.com/wshuyi/my_public_datasets/raw/master/yelp_sample_balanced.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wshuyi/my_public_datasets/master/yelp_sample_balanced.zip [following]
--2019-07-21 21:04:25--  https://raw.githubusercontent.com/wshuyi/my_public_datasets/master/yelp_sample_balanced.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2899278 (2.8M) [application/zip]
Saving to: ‘yelp_sample_balanced.zip’

yelp_sample_balance 100%[===================>]   2.76M  --.-KB/s    in 0.09s   

2019-07-21 21:04:26 (30.7 MB/s) - ‘yelp_sample_balanced.zip’ save

In [3]:
!unzip yelp_sample_balanced.zip

Archive:  yelp_sample_balanced.zip
   creating: yelp_sample_balanced/
  inflating: yelp_sample_balanced/valid.csv  
  inflating: yelp_sample_balanced/test.csv  
  inflating: yelp_sample_balanced/train.csv  


In [0]:
path = Path("yelp_sample_balanced/")

In [5]:
train = pd.read_csv(path/"train.csv"); train.head()

,label,text
0,1,Stopped by here because I was staying at THEho...
1,0,Solid food but the service is lacking.\n\nWe w...
2,1,I won comped tickets to see this show from ESP...
3,0,I truly went with high expectations. I love BB...
4,0,it simply sucks!\n\ni remember the place selli...


In [6]:
print(train.iloc[0, 1])

Stopped by here because I was staying at THEhotel@Mandalay & needed food in mah belly before going out for a night of clubbing ! Wasn't expecting much but left pleasantly surprised.\n\nI got a Buffalo Burger w/ sprouts & avocado -- might as well try something different right ? I was apprehensive at first but WOW my mind was blown! Tender, juicy and came in a really fat patty mmhmmmm. They have a pretty large selection so if you're indecisive you're kinda screwed. Since it's a burger 'bar' you can build your own  burger & customize what you put on it, and they had a pretty good variety of choices.. including truffles or FOIE GRAS if you're feeling fancy!\n\nTheir sweet potato fries & fat fries were pretty good, but he winner of the night was the mound of ZUCCHINI FRIES ! I was setting myself up for a heart attack but it's Vegas, baby and I didn't care !\n\nThey also have a GREAT beer & wine selection, our table got the chimay & fruli  & they were both delicious ! I prefer the chimay tho

In [0]:
valid = pd.read_csv(path/"valid.csv")
test = pd.read_csv(path/"test.csv")

In [8]:
len(train)

6000

In [9]:
len(valid)

2000

In [10]:
len(test)

2000

In [0]:
data_lm = TextLMDataBunch.from_df(path, train, valid, test)

In [12]:
data_lm.train_ds

LabelList (6000 items)
x: LMTextList
xxbos xxmaj stopped by here because i was staying at xxunk & needed food in mah belly before going out for a night of clubbing ! xxmaj was n't expecting much but left pleasantly surprised . 
 
  i got a xxmaj buffalo xxmaj burger w / sprouts & avocado -- might as well try something different right ? i was xxunk at first but xxup wow my mind was blown ! xxmaj tender , juicy and came in a really fat patty xxunk xxrep 4 m . xxmaj they have a pretty large selection so if you 're xxunk you 're kinda screwed . xxmaj since it 's a burger ' bar ' you can build your own burger & customize what you put on it , and they had a pretty good variety of choices .. including truffles or xxup foie xxup gras if you 're feeling fancy ! 
 
  xxmaj their sweet potato fries & fat fries were pretty good , but he winner of the night was the mound of xxup zucchini xxup fries ! i was setting myself up for a heart attack but it 's xxmaj vegas , baby and i did n't care ! 
 
  x

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
data_clas.save('data_clas_export.pkl')

In [15]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.247279,3.964836,0.269315,01:34


In [16]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.890546,3.814734,0.286642,01:55


In [17]:
learn.predict("This is a review about", n_words=10)

'This is a review about my business or business . We had a breakfast'

In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [20]:
data_clas.show_batch()

text,target
xxbos xxup avoid xxup xxunk xxup that xxup works xxup at xxup the xxup body xxup shop ! xxup just xxup dont xxup go xxup there . xxup you xxup will xxup hate xxup yourself xxup if xxup you xxup use xxup them ! xxup just xxup dont ! \n \n xxmaj insurance adjuster from xxmaj farmers ( our insurance company ) came to look at our car and,0
xxbos xxmaj memorial xxmaj day weekend 2013 - xxmaj aoki / xxmaj deadmau5 \n \n xxmaj pre - sale tickets male $ 100 / xxmaj female $ 40 \n \n i had high hopes for xxmaj hakkasan . i went with my girlfriend and we were disappointed with everything except the music . xxmaj steve xxmaj aoki and xxmaj deadmau5 were the only thing that saved the night,0
"xxbos xxmaj the xxmaj wynn is a nice property , a little too far down xxmaj the xxmaj strip to be considered a prime location . xxmaj having said that , it will change in time . xxmaj if xxmaj echelon xxmaj place ( a xxmaj boyd xxmaj gaming project ) and some of the other projects that were once xxunk for development come to xxunk , xxmaj wynn will",1
"xxbos xxmaj went for my birthday , again lol \n \n xxmaj they opened recently , it 's an all you can eat korean bbq place . xxmaj the service is a bit xxunk so expect to wait a bit so do n't go if you 're in a rush . \n \n xxmaj when we were there they had some problems with the ventilation , so expect",1
"xxbos i 've had xxmaj direct xxup tv for about 4 years . i am very , xxup very disappointed in their services . xxmaj when we first got them , we got an amazing deal , so we really did n't want to let it go . xxmaj however , when we first got them we were signed on to a 2 year contract with them . \n \n",0


In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.369455,0.249212,0.904500,01:25


In [22]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.314521,0.207819,0.925500,01:18


In [23]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.263891,0.184232,0.921500,02:26


In [0]:
learn.export("model_trained.pkl")

In [0]:
with open(path/"test_labels.pkl", 'wb') as f:
  pickle.dump(test.label, f)